In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [11]:
data = pd.read_csv('./spotify_data.csv')

In [12]:
data = data.drop(['artists','count','genres'], axis=1)

KeyError: "['count', 'genres'] not found in axis"

# outlier detection

In [ ]:
plt.figure(figsize=(16, 10))

for i in range(1,len(data.columns)):
    plt.subplot(3, 5, i+1)
    sns.boxplot(data[data.columns[i]])

plt.show()

get the outliers after a certain threshold

In [13]:
def get_outlier_counts(df, threshold):
    df = df.copy()
    
    # Get the z-score for specified threshold
    threshold_z_score = stats.norm.ppf(threshold)
    
    # Get the z-scores for each value in df
    z_score_df = pd.DataFrame(np.abs(stats.zscore(df)), columns=df.columns)
    
    # Compare df z_scores to the threshold and return the count of outliers in each column
    return (z_score_df > threshold_z_score).sum(axis=0)

In [14]:
get_outlier_counts(data, 0.9999999)

C:\Users\utilisateur\anaconda3\lib\site-packages\scipy\stats\_stats_py.py:110: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "


TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [ ]:
def remove_outliers(df, threshold):
    df = data.drop(['valence'], axis=1).copy()
    #genres= data['genres'].copy()

    # Get the z-score for specified threshold
    threshold_z_score = stats.norm.ppf(threshold)
    
    # Get the z-scores for each value in df
    z_score_df = pd.DataFrame(np.abs(stats.zscore(df)), columns=df.columns)
    z_score_df = z_score_df > threshold_z_score
    
    # Get indices of the outliers
    outliers = z_score_df.sum(axis=1)
    outliers = outliers > 0
    outlier_indices = df.index[outliers]

    # Drop outlier examples
    df = df.drop(outlier_indices, axis=0).reset_index(drop=True)
    #data3 = genres.drop(outlier_indices, axis=0).copy().reset_index(drop=True)
    #df['genres'] = data3
    return df

In [ ]:
remove_outliers(data, 0.9999999)

# preprocessing

In [ ]:
def preprocess_inputs(df, outliers=True, threshold=0.95):
    #data1 = df.drop(['genres'], axis=1).copy()
    df = df.copy()

    # Remove outliers if specified
    if outliers == False:
        df = remove_outliers(df, threshold)
    
    # Split df into X and y
    y = df['valence'].copy()
    X = df.drop('valence', axis=1).copy()
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=1)
    
    # Scale X with a standard scaler
    scaler = StandardScaler()
    scaler.fit(X_train)
    
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test

In [ ]:
outlier_X_train, outlier_X_test, outlier_y_train, outlier_y_test = preprocess_inputs(data, outliers=True)

X_train, X_test, y_train, y_test = preprocess_inputs(data, outliers=False, threshold=0.9999999999908)

# training

In [ ]:
# With outliers

outlier_model = LinearRegression()
outlier_model.fit(outlier_X_train, outlier_y_train)

outlier_model_acc = outlier_model.score(outlier_X_test, outlier_y_test)

print("Test Accuracy (Outliers): {:.5f}%".format(outlier_model_acc * 100))

In [ ]:
# Without outliers

model = LinearRegression()
model.fit(X_train, y_train)

model_acc = model.score(X_test, y_test)

print("Test Accuracy (No Outliers): {:.5f}%".format(model_acc * 100))

<h1> conclusion </h1>
<h4> after trying a lot of threshold values we can not surpass the (78.34197%) so we assume that all the data is important and we will keep it without deleting outlayers </h4>